In [1]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

# Setting a seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Load Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

# Load data
df = pd.read_csv('Data/Annotations_final.csv')

# Preprocessing steps (if any)

# Grouping data for cross-validation
group_dict = dict()
groups = np.array([])

for index, row in df.iterrows():
    s_id = row['created_by']
    if s_id not in group_dict:
        group_dict[s_id] = index
    groups = np.append(groups, group_dict[s_id])

# Set up the splitter
gkf = GroupKFold(n_splits=5)

# Extract features and labels
X = df['annotation_text']
y = df['comment_answer']

# Storage for performance metrics
roc_auc_scores = np.array([])

# SMOTE for balancing classes
smote = SMOTE(random_state=SEED)

# Model Training and Evaluation
for train_index, test_index in gkf.split(X, y, groups=groups):
    try:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Convert texts to embeddings
        training_embeddings = embed(X_train.to_list())
        testing_embeddings = embed(X_test.to_list())

        # Apply SMOTE
        X_sm, y_sm = smote.fit_resample(training_embeddings.numpy(), y_train)

        # Define the model
        model = Sequential([
            Dense(12, activation='relu', input_shape=(512,)),
            Dense(8, activation='relu'),
            Dense(1, activation='sigmoid')
        ])

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # Model training
        model.fit(X_sm, y_sm, epochs=30, validation_split=0.1, shuffle=True, batch_size=10)

        # Model evaluation
        predictions = model.predict(testing_embeddings)
        roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))
    except Exception as e:
        print(f"An error occurred: {e}")

# Output the results
print(f"Average ROC AUC Score: {np.round(roc_auc_scores.mean(), 3)}")
print(f"ROC AUC Score Standard Deviation: {np.round(roc_auc_scores.std(), 3)}")
print(f"Maximum ROC AUC Score: {np.round(roc_auc_scores.max(), 3)}")


Epoch 1/30



30/30 [==============================] - 1s 5ms/step - loss: 0.6808 - accuracy: 0.7216 - val_loss: 0.6364 - val_accuracy: 1.0000
Epoch 2/30
30/30 [==============================] - 0s 2ms/step - loss: 0.6245 - accuracy: 0.9003 - val_loss: 0.5114 - val_accuracy: 1.0000
Epoch 3/30
30/30 [==============================] - 0s 2ms/step - loss: 0.5192 - accuracy: 0.9141 - val_loss: 0.3307 - val_accuracy: 1.0000
Epoch 4/30
30/30 [==============================] - 0s 2ms/step - loss: 0.4025 - accuracy: 0.9347 - val_loss: 0.2495 - val_accuracy: 1.0000
Epoch 5/30
30/30 [==============================] - 0s 2ms/step - loss: 0.3033 - accuracy: 0.9725 - val_loss: 0.1822 - val_accuracy: 1.0000
Epoch 6/30
30/30 [==============================] - 0s 2ms/step - loss: 0.2300 - accuracy: 0.9794 - val_loss: 0.1335 - val_accuracy: 1.0000
Epoch 7/30
30/30 [==============================] - 0s 2ms/step - loss: 0.1744 - accuracy: 0.9828 - val_loss: 0.1372 - val_accuracy: 1.0000
Epoch 8/30
30/30 [=============

2/2 [==============================] - 0s 2ms/step
Average ROC AUC Score: 0.965
ROC AUC Score Standard Deviation: 0.021
Maximum ROC AUC Score: 0.988
